# ============================================
# LangGraph – Annotations & State Typing
# ============================================
**Author:** Dr. Dasha Trofimova

### Goals
- Refresh **Python type annotations** for ML systems
- Define a **typed State** for LangGraph with `TypedDict`
- Add **docstrings** and human-readable descriptions to nodes
- Validate and trace state during execution

---


In [ ]:
# Optional installs (LangGraph for later cells)
!pip install -q langgraph mypy_extensions

## 1) Python typing refresher

Common forms you'll use in LangGraph projects:
- `TypedDict` — define a dictionary with fixed keys and value types (great for State)
- `Literal` — restrict to specific string values (e.g., routing tags)
- `Optional[T]` — value may be `None`
- `Annotated[T, ...]` — add metadata (e.g., validators, descriptions)

We'll demonstrate each briefly.


In [ ]:
from typing import TypedDict, Optional, Literal, Annotated, List, Dict, Tuple

class UserMsg(TypedDict):
    role: Literal["user", "system"]
    text: str
    tokens: Optional[int]

def word_count(s: str) -> int:
    return len(s.split())

MetaInt = Annotated[int, "non-negative", "counter-like"]
def add(a: MetaInt, b: MetaInt) -> MetaInt:
    return a + b

# Demo
u: UserMsg = {"role": "user", "text": "hello class", "tokens": None}
print("UserMsg:", u)
print("word_count:", word_count(u["text"]))
print("add(2,3):", add(2,3))

## 2) Typed State for LangGraph

We'll define a **State** that flows through the graph.

Example: a small pipeline that builds a status string:
- `name` *(str)*
- `age` *(int)*
- `ok` *(bool)*
- `status` *(str)*


In [ ]:
from typing import TypedDict

class State(TypedDict):
    name: str
    age: int
    ok: bool
    status: str

s: State = {"name": "Alex", "age": 23, "ok": True, "status": ""}
s

## 3) Adding node annotations & docstrings

We'll create nodes (plain Python functions) and attach **docstrings** and a small `NODE_DESC` registry for clarity.


In [ ]:
from langgraph.graph import StateGraph, END

NODE_DESC = {
    "check_age": "Validates age >= 18 and sets ok=True/False.",
    "format_status": "Formats a readable status string from the state."
}

def describe(name):
    def deco(fn):
        fn.__doc__ = NODE_DESC.get(name, fn.__doc__ or "")
        def wrap(st: State) -> State:
            print(f"\n▶ {name}: {fn.__doc__}")
            out = fn(st)
            print("   in:", st)
            print("   out:", out)
            return out
        return wrap
    return deco

@describe("check_age")
def check_age(st: State) -> State:
    return {**st, "ok": bool(st["age"] >= 18)}

@describe("format_status")
def format_status(st: State) -> State:
    msg = f"{st['name']} ({st['age']} yrs) is " + ("an adult" if st["ok"] else "a minor")
    return {**st, "status": msg}

g = StateGraph(State)
g.add_node("check_age", check_age)
g.add_node("format_status", format_status)
g.add_edge("check_age", "format_status")
g.add_edge("format_status", END)
g.set_entry_point("check_age")

app = g.compile()
print(app.invoke({"name": "Alex", "age": 23, "ok": False, "status": ""})["status"])

## 4) Optional: light validation and helpful errors

Add simple checks at node boundaries to catch bad state early.


In [ ]:
def require_keys(st: dict, keys):
    missing = [k for k in keys if k not in st]
    if missing:
        raise KeyError(f"State missing required keys: {missing}")

@describe("check_age")
def check_age_validated(st: State) -> State:
    require_keys(st, ["age"])
    return {**st, "ok": bool(st["age"] >= 18)}

# Rebuild graph with validated node
g2 = StateGraph(State)
g2.add_node("check_age", check_age_validated)
g2.add_node("format_status", format_status)
g2.add_edge("check_age", "format_status"); g2.add_edge("format_status", END)
g2.set_entry_point("check_age")
print(g2.compile().invoke({"name":"Sam","age":17,"ok":False,"status":""})["status"])

### ✅ Takeaways
- Use `TypedDict` to make your **State** explicit and self-documenting
- Keep a **NODE_DESC** map and docstrings so new readers understand each node
- Add lightweight **validation** where it helps catch mistakes early

---


### 🎯 Quick Card Quiz — Annotations
- **White** = `TypedDict`
- **Brown** = `Literal`
- **Green** = `Annotated`

1) Which one lets you define a dictionary with fixed keys used as State?
2) Which one restricts values to a fixed set (e.g., `'calc'|'echo'`)?
3) Which one attaches metadata (e.g., "counter-like") to a type?
